In [574]:
from BeautifulSoup import BeautifulSoup
import bs4
import json
import requests
import argparse
import csv
import operator
import codecs
import cStringIO
import urllib2
import nltk
import sys
import movie_scrapers

# STEP 1: Generate List of Animated Movies Released Since 1980

In [576]:
test_wiki_scraper = scrape_movie_lists('https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2016')
print '\n'.join(test_wiki_scraper[0:10])

Accel World: Infinite Burst
Ajin -Shōtotsu-
Ajin: Shōgeki
Albert
The Angry Birds Movie
Another Day of Life
The Ape Story
Ballerina
Big Fish & Begonia
BoBoiBoy: The Movie


In [577]:
# To start, I will look at all animated films released in 1980 or later:
# It's faster to copy paste the link and just change the last digits to reflect the release year:

urls = ["https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1980",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1981",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1982",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1983",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1984",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1985",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1986",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1987",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1988",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1989",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1990",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1991",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1992",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1993",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1994",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1995",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1996",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1997",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1998",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_1999",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2000",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2001",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2002",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2003",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2004",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2005",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2006",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2007",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2008",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2009",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2010",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2011",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2012",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2013",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2014",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2015",
       "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2016"]

In [5]:
# Call the function scrape_movie_lists for each year, starting with 1980:

movie_titles_all = []
for url in urls:
    movie_titles_all.append(scrape_movie_lists(url))
movie_titles_all = reduce(operator.concat, movie_titles_all)
print len(movie_titles_all)

2633


In [81]:
print '\n'.join(movie_titles_all[0:10])

15 Sonyeon Uju Pyoryugi
Animalympics
Be Forever Yamato
Bloody Lady
Bon Voyage, Charlie Brown (and Don't Come Back!!)
Cyborg 009: Legend of the Super Galaxy
Doksuri 5 Hyeongje
Doraemon: The Motion Picture
Eleven Hungry Cats
Fumoon


In [148]:
# Copied from python documentation:

class UnicodeWriter:
    """
    A CSV writer which will write rows to CSV file "f",
    which is encoded in the given encoding.
    """

    def __init__(self, f, dialect=csv.excel, encoding="utf-8", **kwds):
        # Redirect output to a queue
        self.queue = cStringIO.StringIO()
        self.writer = csv.writer(self.queue, dialect=dialect, **kwds)
        self.stream = f
        self.encoder = codecs.getincrementalencoder(encoding)()

    def writerow(self, row):
        self.writer.writerow([s.encode("utf-8") for s in row])
        # Fetch UTF-8 output from the queue ...
        data = self.queue.getvalue()
        data = data.decode("utf-8")
        # ... and reencode it into the target encoding
        data = self.encoder.encode(data)
        # write to the target stream
        self.stream.write(data)
        # empty queue
        self.queue.truncate(0)

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
        

In [348]:
# Save the list of animated movies in a csv file:

with open('movie_list.csv', 'wb') as output:
    writer = UnicodeWriter(output, lineterminator='\n', quoting=csv.QUOTE_NONNUMERIC)
    for val in movie_titles_all:
        writer.writerow([val])  

# STEP 2: Scrape IMDB for Full Cast and Crew, Year of Release, and Rating

 The hyperlink I want looks like: http://www.imdb.com/title/tt2096673/fullcredits?ref_=tt_cl_sm#cast

In [146]:
# Call the function get_imdb_links for each movie: 

imdb_urls = []
current_idx = 0
m = len(movie_titles_all)
for movie in movie_titles_all:
    imdb_urls.append(get_imdb_links(movie))
    
    if (current_idx % 10) == 0:
        progress = 100.0 * current_idx / m
        sys.stdout.write('\r progress {} / {} ({:6.2f}%)'.format(current_idx, m, progress))
        sys.stdout.flush()
    current_idx += 1
        
print len(imdb_urls)

 progress 2630 / 2633 ( 99.89%)2633


In [150]:
# Save the list of imdb movie links in a csv file:

with open('imdb_url_list.csv', 'wb') as output:
    writer = UnicodeWriter(output, lineterminator='\n', quoting=csv.QUOTE_NONNUMERIC)
    for val in imdb_urls:
        if val:
            val = val[0:17]
            writer.writerow([val])  

Note that the length of the CSV file is only 1922 rows. This indicates that ~27% of the movies released and published to wikipedia were not on IMDB. I spot checked a few movies, and most of the missing 27% were foreign animated movies.

In [572]:
class Movie:
        
    def __init__(self, url_key):
        
        # Get movie metrics by scraping the main IMDB movie page:
        imdb_url = "http://www.imdb.com"+url_key
    
        response = requests.get(imdb_url)
        soup = bs4.BeautifulSoup(response.text, "html.parser")
        title = soup.findAll("div","title_block")[0]
        rating = title.find("strong").get_text()
        num_ratings = title.find("div","imdbRating").a.get_text()
        release_year = title.find("div","title_wrapper").a.get_text()
        run_time = title.find("time").get_text()
        
        # Get full cast and crew by scraping the IMDB movie credits page:
        imdb_credits_url = "http://www.imdb.com"+url_key+"fullcredits?ref_=tt_cl_sm#cast"
    
        response_credits = requests.get(imdb_credits_url)
        credits_soup = bs4.BeautifulSoup(response_credits.text, "html.parser")
        credits_data = credits_soup.findAll("div","header")[0]
        
        # Store movie credits in a list of tuples of the form (name,role):
        full_credits = []

        # Store movie credits in a list of tuples of the form (name, team):
        credits_by_teams = []
        teamnames = []

        find_teamnames = credits_data.findAll("h4","dataHeaderWithBorder")
        for teamname in find_teamnames:
            teamnames.append(teamname.get_text())
        
        team_tables = credits_data.findAll("table")
        for idx,table in enumerate(team_tables):
            crew_byrow = table.findAll("tr")
        
            for row in crew_byrow:
                tds = row.findAll('td')
        
                for td in tds:
                    if td.a:
                        name = td.a.get_text()
                
                if row.findAll("td","credit"):
                    role = row.findAll("td","credit",limit=1)[0].get_text()
        
                    full_credits.append((name,role))
                    credits_by_teams.append((name, teamnames[idx]))
                  
        self.rating = rating
        self.num_ratings = num_ratings
        self.release_year = release_year
        self.run_time = run_time
        self.full_credits = full_credits
        self.credits_by_teams = credits_by_teams
        
    def get_team_members(self, teamname):
        return [name for name,teamnames in self.credits_by_teams if teamname in teamnames]

In [578]:
random_movie = Movie('/title/tt0114709/')
print random_movie.get_team_members("Writing Credits")

[u' John Lasseter\n', u' Pete Docter\n', u' Andrew Stanton\n', u' Joe Ranft\n', u' Joss Whedon\n', u' Andrew Stanton\n', u' Joel Cohen\n', u' Alec Sokolow\n']


In [558]:
def get_movie_metrics(url_key):
    """
    This function scrapes IMDB and returns various metrics for
    movie of interest (including year of release, IMDB rating, 
    estimated budget, gross, and run time). 
    """
    
    imdb_url = "http://www.imdb.com"+url_key
    
    response = requests.get(imdb_url)
    soup = bs4.BeautifulSoup(response.text, "html.parser")
    title = soup.findAll("div","title_block")[0]
    
    rating = title.find("strong").get_text()
    num_ratings = title.find("div","imdbRating").a.get_text()
    release_year = title.find("div","title_wrapper").a.get_text()
    run_time = title.find("time").get_text()
    
    find_bottom_pg_data = soup.findAll("div","article","titleDetails")
    
    for data in find_bottom_pg_data:
        find_box_office = data.findAll("div")
        for entry in find_box_office:
            budget = entry.find("itemprop")
       
    return budget

In [559]:
print get_movie_metrics('/title/tt0114709/')

None
